# Scraping Process

## Importing the Library

In [ ]:
import pandas as pd 
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

## Make the driver and Loading the addresss

In [ ]:
driver = webdriver.Chrome()
url = 'https://www.travelio.com/search?searchType=monthly&destinationCategory=City&destinationUrlName=&destinationPlaceId=&destinationCountryId=ID&destinationId=544a05f190e50d6a3d000001&nights=30&flexible=1&destination=Jakarta&checkIn=01-04-2023&checkOut=01-05-2023&months=1&propTypeId=apartment&cbFlexible=on&unitType=3%2C2%2C1%2Cstudio&sellType=Unfurnished%2CFull%2BFurnished&bottomPrice=0&upperPrice=20000000'
driver.get(url)
# keep track of the scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # wait for the page to load
    time.sleep(5)
    
    # calculate the new scroll height and compare to the previous scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # if the new scroll height is the same as the previous scroll height, we've reached the end of the page
    if new_height == last_height:
        break
    
    # update the last scroll height
    last_height = new_height

## Scrap the url for each unit page

In [ ]:
unit_selections = driver.find_elements(By.CSS_SELECTOR, '.property-box a')
unit_link = []
for i in unit_selections:
    url = i.get_attribute('href')
    unit_link.append({'url': url})

unit_link_df = pd.DataFrame(unit_link)
unit_link_df

## Scrap the Data

In [ ]:
def get_text_or_default(driver, xpath, default='N/A'):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.text
    except NoSuchElementException:
        return default

unit_df = []

for m in range(0,len(unit_link_df)):
    driver.get(unit_link_df['url'][m])
    
    button_appeared = False
    while not button_appeared:
        try:
            # Wait for the close button to be clickable
            wait = WebDriverWait(driver, 15)
            close_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'close-button')))
            
            # Click the close button
            close_button.click()
            time.sleep(2)

            button_appeared = True
            
        except TimeoutException:
            # Reload the page if the button hasn't appeared within 30 seconds
            driver.refresh()
    # Get all element that we want        
    unit_name = get_text_or_default(driver, '//*[@id="hotel-name"]/div[1]/h2')
    furnish_status = get_text_or_default(driver, '//*[@id="hotel-name"]/div[2]/span[1]')
    num_rooms = get_text_or_default(driver, '//*[@id="hotel-name"]/div[2]/span[2]')
    num_bathroom = get_text_or_default(driver, '//*[@id="hotel-name"]/div[2]/span[3]')
    elec_power = get_text_or_default(driver, '//*[@id="hotel-name"]/div[2]/span[4]')
    location = get_text_or_default(driver, '//*[@id="hotel-address"]/span[1]')
    size = get_text_or_default(driver, '//*[@id="hotel-room-detail"]/div[2]/div[1]/div[2]')
    room_type = get_text_or_default(driver, '//*[@id="hotel-room-detail"]/div[2]/div[3]/div[2]')
    property_type = get_text_or_default(driver, '//*[@id="page-body-left"]/div[5]/div[2]/div[1]/div[2]')
    building = get_text_or_default(driver, '//*[@id="page-body-left"]/div[5]/div[2]/div[2]/div[1]/div[2]')
    tower =  get_text_or_default(driver, '//*[@id="page-body-left"]/div[5]/div[2]/div[2]/div[2]/div[2]')
    floor_num = get_text_or_default(driver, '//*[@id="page-body-left"]/div[5]/div[2]/div[2]/div[3]/div[2]') 
    room_facility = get_text_or_default(driver, '//*[@id="page-body-left"]/div[5]/div[3]/div/div[2]/div[2]')
    kitchen_facility = get_text_or_default(driver, '//*[@id="page-body-left"]/div[5]/div[3]/div/div[2]/div[4]')
    building_facility = get_text_or_default(driver, '//*[@id="page-body-left"]/div[5]/div[4]/div/div[2]')
    clean_rating = get_text_or_default(driver, '//*[@id="review-list-container"]/div[2]/div[1]/div[3]')
    location_rating = get_text_or_default(driver, '//*[@id="review-list-container"]/div[2]/div[3]/div[3]')
    satis_rating = get_text_or_default(driver, '//*[@id="review-list-container"]/div[2]/div[5]/div[3]')
    check_rating = get_text_or_default(driver, '//*[@id="review-list-container"]/div[2]/div[2]/div[3]')
    econom_rating = get_text_or_default(driver, '//*[@id="review-list-container"]/div[2]/div[2]/div[3]')
    staff_rating = get_text_or_default(driver, '//*[@id="review-list-container"]/div[2]/div[4]/div[3]')
    total_rating = get_text_or_default(driver, '//*[@id="property-rate"]/b')
    longitude = driver.find_element(By.XPATH, '//*[@id="longitude"]')
    latitude = driver.find_element(By.XPATH, '//*[@id="latitude"]')
    longitude_value = longitude.get_attribute('value')
    latitude_value = latitude.get_attribute('value')
    monthly_price = get_text_or_default(driver, '//*[@id="price-calculation"]/div[1]/div[2]') 
    
    unit_df.append({'unit_name': unit_name,
                    'furnish_status': furnish_status,
                    'num_rooms': num_rooms,
                    'num_bathroom': num_bathroom,
                    'elec_power': elec_power,
                    'location': location,
                    'longitude': longitude_value,
                    'latitude': latitude_value,
                    'size': size,
                    'room_type': room_type,
                    'property_type': property_type,
                    'building': building,
                    'tower': tower,
                    'floor_num': floor_num,
                    'room_facility': room_facility,
                    'kitchen_facility': kitchen_facility,
                    'building_facility': building_facility,
                    'clean_rating': clean_rating,
                    'location_rating': location_rating,
                    'satisfaction_rating': satis_rating,
                    'checkin/out_rating': check_rating,
                    'economic_rating': econom_rating,
                    'staff_rating': staff_rating, 
                    'total_rating': total_rating,
                    'monthly_price': monthly_price})


# Data Cleaning process

## Convert to Data Frame

In [ ]:
pd.options.display.max_columns = None  

In [ ]:
df = pd.DataFrame(unit_df)

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
try_df = df[['num_bathroom','elec_power']]
try_df

In [ ]:
try_df['elec_power'] = try_df.apply(lambda row: row['num_bathroom'] if row['elec_power'] == 'N/A' else row['elec_power'], axis=1)

In [ ]:
df['elec_power'] = try_df['elec_power']

In [ ]:
try_df = df[['num_rooms','num_bathroom']]
import re

# Define a regular expression pattern to match numeric values
pattern = r'\d+'

# Define a function to extract the numeric portion of a string and convert it to an integer
def extract_number(s):
    match = re.search(pattern, s)
    if match:
        return int(match.group())
    else:
        return None

# Apply the function to the 'num_bathroom' column to extract the numeric values
try_df['num_bathroom'] = try_df['num_bathroom'].apply(extract_number)

try_df['num_bathroom'] = try_df.apply(lambda x: x['num_rooms'] if x['num_bathroom'] not in [1, 2, 3] else x['num_bathroom'], axis=1)

try_df

In [ ]:
try_df['num_bathroom'].value_counts()

In [ ]:
df['num_bathroom'] = try_df['num_bathroom']
df

In [ ]:
df['elec_power'] = df['elec_power'].str.replace('Watt', '')
df

In [ ]:
df = df.replace('N/A', np.nan)
df.isnull().sum()

In [ ]:
# convert to numeric
df['num_rooms'] = pd.to_numeric(df['num_rooms'], errors='coerce')
df['num_bathroom'] = pd.to_numeric(df['num_bathroom'], errors='coerce')
df['elec_power'] = pd.to_numeric(df['elec_power'], errors='coerce')

# fill null values
df['num_rooms'] = df['num_rooms'].fillna(value=np.nan)
df['num_bathroom'] = df['num_bathroom'].fillna(value=np.nan)
df['elec_power'] = df['elec_power'].fillna(value=np.nan)

In [ ]:
df

In [ ]:
df.fillna(value=np.nan, inplace=True)
df.isnull().sum()

In [ ]:
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)
df

In [ ]:
df['size'] = df['size'].astype(float)
df

In [ ]:
# Select rows that meet the condition
mask = ~df['property_type'].str.contains('Apartemen', na=False)
df.loc[mask, 'property_type'] = 'Apartemen'

In [ ]:
def extract_max_number(val):
    if pd.isna(val):
        return None
    elif isinstance(val, int):
        val = str(val)
    if ',' in val:
        num_list = val.split(',')
        try:
            return max(list(map(int, num_list)))
        except ValueError:
            return None
    else:
        try:
            return int(val)
        except ValueError:
            return None

In [ ]:
df['floor_num'] = df['floor_num'].apply(extract_max_number)
df['floor_num'] = df['floor_num'].fillna(np.nan)
df['floor_num'].unique()

In [ ]:
df.loc[df['floor_num'] == 2102.0] 

In [ ]:
df['floor_num'][2263] = 21

In [ ]:
def clean_price(price_str):
    if price_str and isinstance(price_str, str):
        price_str = price_str.replace(',', '').replace('IDR', '').strip()
        return int(price_str)
    else:
        return None

df['monthly_price'] = df['monthly_price'].apply(clean_price)

In [ ]:
null_monthly_price = df[df['monthly_price'].isnull()]
null_monthly_price

In [ ]:
df['monthly_price'][304] = 4209787
df['monthly_price'][327] = 3620000
df['monthly_price'][444] = 3820000
df['monthly_price'][747] = 4121713
df['monthly_price'][845] = 4950659
df['monthly_price'][978] = 4665000
df['monthly_price'][1274] = 5256285
df['monthly_price'][1310] = 5321706
df['monthly_price'][1602] = np.nan
df['monthly_price'][1715] = 6620000
df['monthly_price'][2176] = np.nan
df['monthly_price'][2319] = 14478600
df['monthly_price'][2377] = 15510000
df['monthly_price'][2391] = 16105270
df['monthly_price'][2421] = np.nan
df['monthly_price'][3047] = 3620000

In [ ]:
df['clean_rating'] = df['clean_rating'].astype(float)
df['location_rating'] = df['clean_rating'].astype(float)
df['satisfaction_rating'] = df['clean_rating'].astype(float)
df['checkin/out_rating'] = df['clean_rating'].astype(float)
df['economic_rating'] = df['clean_rating'].astype(float)
df['staff_rating'] = df['clean_rating'].astype(float)
df['total_rating'] = df['clean_rating'].astype(float)

In [ ]:
df = df.replace('\n', ',', regex=True)

In [ ]:
df

In [ ]:
df.to_csv('unit_df.csv')